In [ ]:
import os.path
from os import path
import numpy as np
import matplotlib.pyplot as pyplot
import gc
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import statistics
import seaborn as sn
import random
import copy
import math
from itertools import permutations

Fill in the following paths that contain:

neutrons_train_path: neutron pulses used for training\
gammas_train_path: gamma pulses used for training\
neutrons_val_path: neutron pulses used for validation\
gammas_val_path: gamma pulses used for validation\
template_n_path: template of the average neutron\
template_g_path: template of the average gamma

Provide the following paths in which values will be saved:

path_model_FC_NN: raw pulse DNN\
path_model_FC_NN_params: raw pulse DNN parameters\
path_model_Concat: features DNN\
path_model_Attention: Deep Classifier\
path_model_Attention_params: features DNN + Deep Classifier parameters\
path_model_BM_FCNN: DNN 6 outputs\
path_model_BM_FCNN_params: DNN 6 outputs parameters\
path_model_BM_FCNN1: Two DNNs\
path_model_BM_FCNN1_params: Two DNNs parameters\
path_CI_parameters: CI parameters\
path_NCC_classifier_parameters: NCC parameters\
classifier_config_path: constants are saved here

In [ ]:
# Parameters:
arrival_rate = 6e6
g_n_ratio = 1

# Constants:
starting_index = 0
template_length_orig_features = 400
Ts = 1e-9
num_original_features = 13
num_classification_features = 8
num_attention_features = 8
new_starting_point = 20
pulse_len_after_starting = 1000
time_offset = 15
success_marg = 15
L_deriv_orig_features = 500
delta_orig_features = 8
short_gate_orig_features = 30
long_gate_orig_features = 100
max_random_offset = 15
cont_starting_time = 10000
original_pulse_len = 9460
std_noise_total = 0.00259578
std_noise_curr_constant = 0.000269
noise_mean = 0
len_true_inf = num_original_features + 9
lambda_n = int(arrival_rate/(1+g_n_ratio))
lambda_g = int((arrival_rate*g_n_ratio)/(1+g_n_ratio))
std_noise_curr = std_noise_curr_constant*np.sqrt(original_pulse_len*arrival_rate*Ts)
std_added_noise = np.sqrt(np.power(std_noise_total,2) - np.power(std_noise_curr,2))

run <your-path> 'General_Functions.ipynb>

Num_train = 100000 # Number of pulses used for training
Num_val = 40000 # Number of pulses used for validation
Num_test = 60000 # Number of pulses used for testing
WL_classifier = 200 # Number of pulse samples used for DNN methods
max_dist = 120
rel_ratio = np.exp(-arrival_rate*max_dist*1e-9)
N_single_train = int(0.5*Num_train*rel_ratio)
N_double_train = int(0.25*Num_train*(1-rel_ratio))
N_single_val = int(N_single_train/5)
N_double_val = int(N_double_train/5)

neutrons_train = np.load(neutrons_train_path)
gammas_train = np.load(gammas_train_path)
neutrons_val = np.load(neutrons_val_path)
gammas_val = np.load(gammas_val_path)
template_n = np.load(template_n_path)
template_g = np.load(template_g_path)

train_data, train_labels = create_classifier_data_for_training(neutrons_train,gammas_train,Num_train,False)

# Deep Classifier and DNN based methods Layers
model_classifier_layer1 = 130
model_classifier_layer2 = 80
model_classifier_layer3 = 30
model_classifier_layer4 = 12
model_classifier_layer5 = 4
model_attention_layer1 = 30
model_attention_layer2 = 80

run <your-path> 'Classification_Algorithm_Functions.ipynb>

## Optimizing the Charge Integration based PSD
CI_param1_lst = range(1,100,1) # Short gate length in samples
CI_param2_lst = range(1,200,1) # Long gate length in samples
CI_param3_lst = range(-30,50,1) # Delay before starting to integrate in samples
CI_param4_lst = list(np.linspace(0.2,1,200)) # Threshold

CI_parameters = optimize_charge_integration_parameters(train_data,train_labels)
np.save(path_CI_parameters,CI_parameters)

## Optimizing the Normalized Cross-Correlation based PSD
corr_param1_lst = range(0,30,1) # Correlation starting index
corr_param2_lst = range(30,200,1) # Correlation ending index

Corr_parameters = optimize_correlation_parameters(train_data,train_labels)
np.save(path_NCC_classifier_parameters,Corr_parameters)

## DNN based methods
train_data, train_labels, train_class_feat, train_weigh_feat = create_classifier_data_for_training(neutrons_train,gammas_train,Num_train,True)
val_data, val_labels, val_class_feat, val_weigh_feat = create_classifier_data_for_training(neutrons_val,gammas_val,Num_val,True)
short_gate_class_features = CI_parameters[0]
long_gate_class_features = CI_parameters[1]
start_gate_class_features = CI_parameters[2]
corr_start_class_features = Corr_parameters[0]
corr_end_class_features = Corr_parameters[1]

## Training the DNN applied to the raw pulse x_t
model_FC_NN_lr = 1e-3
model_FC_NN_bs = 30
model_FC_NN_epochs = 70
run_Model_FC_NN(train_labels,train_data,val_labels,val_data,val_labels,
                val_data,path_model_FC_NN,
                path_model_FC_NN_params, model_FC_NN_lr, model_FC_NN_bs, model_FC_NN_epochs)

## Training the Deep Classifier and the DNN applied to both the raw pulse x_t and classical features y_t
model_Attention_lr = 1e-3
model_Attention_bs = 30
model_Attention_epochs = 70
run_Model_Attention(train_labels,train_data,val_labels,val_data,
                val_labels,val_data,train_class_feat,val_class_feat,val_class_feat,
                train_weigh_feat,val_weigh_feat,val_weigh_feat,path_model_Concat,path_model_Attention,
                path_model_Attention_params, model_Attention_lr, model_Attention_bs, model_Attention_epochs)


train_data, train_labels = create_BM_FC_data_for_training(neutrons_train,gammas_train,N_single_train,N_double_train)
val_data, val_labels = create_BM_FC_data_for_training(neutrons_val,gammas_val,N_single_val,N_double_val)
train_data_n1, train_labels_n1, train_data_n2, train_labels_n2 = split_data_for_new_bm(train_data, train_labels,N_single_train,N_double_train)
val_data_n1, val_labels_n1, val_data_n2, val_labels_n2 = split_data_for_new_bm(val_data, val_labels,N_single_val,N_double_val)

## Training the DNN with 6 output neurons: n,g,nn,ng,gn,gg
model_BM_FCNN_lr = 1e-3
model_BM_FCNN_bs = 30
model_BM_FCNN_epochs = 140
run_Model_BM_FCNN(train_labels,train_data,val_labels,val_data,val_labels,
                val_data,path_model_BM_FCNN,
                path_model_BM_FCNN_params, model_BM_FCNN_lr, model_BM_FCNN_bs, model_BM_FCNN_epochs)

## Training the method utilizing two separate DNNs, the first for n,g and the second for nn,ng,gn,gg
model_BM_FCNN1_lr = 1e-3
model_BM_FCNN1_bs = 30
model_BM_FCNN1_epochs = 140
run_Model_BM_FCNN1(train_labels_n1,train_data_n1,val_labels_n1,val_data_n1,val_labels_n1,
                val_data_n1,path_model_BM_FCNN1,
                path_model_BM_FCNN1_params, model_BM_FCNN1_lr, model_BM_FCNN1_bs, model_BM_FCNN1_epochs)

constants_dict = {
    "arrival_rate": arrival_rate,
    "g_n_ratio": g_n_ratio,

    "starting_index": starting_index,
    "template_length_orig_features": template_length_orig_features,
    "Ts": Ts,
    "num_original_features": num_original_features,
    "num_classification_features": num_classification_features,
    "num_attention_features": num_attention_features,
    "new_starting_point": new_starting_point,
    "pulse_len_after_starting": pulse_len_after_starting,
    "time_offset": time_offset,
    "success_marg": success_marg,
    "L_deriv_orig_features": L_deriv_orig_features,
    "delta_orig_features": delta_orig_features,
    "short_gate_orig_features": short_gate_orig_features,
    "long_gate_orig_features": long_gate_orig_features,
    "max_random_offset": max_random_offset,
    "cont_starting_time": cont_starting_time,
    "original_pulse_len": original_pulse_len,
    "std_noise_total": std_noise_total,
    "std_noise_curr_constant": std_noise_curr_constant,
    "noise_mean": noise_mean,
    "len_true_inf": len_true_inf,
    "lambda_n": lambda_n,
    "lambda_g": lambda_g,
    "std_noise_curr": std_noise_curr,
    "std_added_noise": std_added_noise,
    "arrival_rate_text": arrival_rate_text,
    "curr_path": curr_path,

    "short_gate_class_features": short_gate_class_features,
    "long_gate_class_features": long_gate_class_features,
    "start_gate_class_features": start_gate_class_features,
    "corr_start_class_features": corr_start_class_features,
    "corr_end_class_features": corr_end_class_features,

    "Num_train": Num_train,
    "Num_val": Num_val,
    "Num_test": Num_test,
    "Num_BM": Num_BM,
    "WL_classifier": WL_classifier,

    "config_path": config_path,
    "model_classifier_layer1": model_classifier_layer1,
    "model_classifier_layer2": model_classifier_layer2,
    "model_classifier_layer3": model_classifier_layer3,
    "model_classifier_layer4": model_classifier_layer4,
    "model_classifier_layer5": model_classifier_layer5,
    "model_attention_layer1": model_attention_layer1,
    "model_attention_layer2": model_attention_layer2,
    "model_FC_NN_lr": model_FC_NN_lr,
    "model_FC_NN_bs": model_FC_NN_bs,
    "model_FC_NN_epochs": model_FC_NN_epochs,

    "model_Attention_lr": model_Attention_lr,
    "model_Attention_bs": model_Attention_bs,
    "model_Attention_epochs": model_Attention_epochs,
}

with open(classifier_config_path, "wb") as file:
    pickle.dump(constants_dict, file)